In [ ]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

gamma = 0.42

# Pad the kernel to match the size of the image
padding = (kernel_x.shape[0] - 1) // 2
kerx_pad = np.pad(kernel_x, padding, 'constant')
kery_pad = np.pad(kernel_y, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s)
def g(s):
    return 1 / (1 + (s**2 + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, less, kerx, kery):
    k_u_x = cv2.filter2D(image, -1, kerx)
    phi_x = k_u_x*g(k_u_x)
    p_x = dt * phi_x
    ans_x = (((2 + gamma)*image) - less + p_x)/(1+gamma)
    
    k_u_y = cv2.filter2D(image, -1, kery)
    phi_y = k_u_y*g(k_u_y)
    p_y = dt * phi_y
    ans_y = (((2 + gamma)*image) - less + p_y)/(1+gamma)
    
    ans = (ans_x + ans_y)/2.0
    return ans

# Load the image
image = cv2.imread('lena_noisy.png', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
img_less = image.copy()
for _ in range(20):
    temp = denoised_image.copy()
    denoised_image = denoise_image(denoised_image, img_less, kerx_pad, kery_pad)
    img_less = temp.copy()

cv2.imwrite('lena_both_perona.jpg', denoised_image * 255.0)